## EEM with drude/shell particles
The EEM equation can be extended to take into account polarizable shell/drude particles (from now on called shells), as described in the Ph.D. thesis by [Mohammad Ghahremanpour](http://urn.kb.se/resolve?urn=urn%3Anbn%3Ase%3Auu%3Adiva-380687), however with some additional justifications. We rewrite the charges to have a superscript c (core) or s (shell), the same superscripts are used for the Coulomb integrals. In the Alexandria models the shell charge is fixed and only core charges can change in these algorithms. The magnitude of the shell charges influences the values of the fited parameters to some extent and is therefore part of the model. That implies that ad-hoc changes to the shell charges invalidate the model.

$E_{EEM}(q_1^c,q_2^c,...,q_N^c) = \sum\limits_{i=1}^{N} \left[ \chi_i \left(q_i^c+q_i^s\right) + \frac{1}{2}\eta_i (q_i^c+q_i^s)^2 + \frac{1}{2}\left(q_i^c\sum\limits_{l \ne i}^{N} q_l^c J_{il}^{cc} + q_i^c\sum\limits_{l \ne i}^{N} q_l^s J_{il}^{cs} + q_i^s\sum\limits_{l \ne i}^{N} q_l^c J_{il}^{sc}  + q_i^s\sum\limits_{l \ne i}^{N} q_l^s J_{il}^{ss}\right) \right]$

The last two terms do no contain $q_i^c$ and therefore drop out of the derivative with respect to the core charge, which becomes:

$0 = \displaystyle{\frac{\partial E_{EEM}}{\partial q_i^c}} = \chi_{eq} = \chi_i + (q_i^c+q_i^s)\eta_i + \frac{1}{2}\left(\sum\limits_{l \ne i}^{N} q_l^c J_{il}^{cc}+ \sum\limits_{l \ne i}^{N}q_l^s J_{il}^{cs}\right) $

This means that we can use the same matrix equation to solve for the core charges as in the EEM without shells, except for that the right hand side of the equation gets two contributions from the shell particles. The right hand side terms $rhs_i$ now become:

$rhs_i = -\chi_i - \eta_i q_i^s - \frac{1}{2}\sum\limits_{l \ne i}^Nq_l^s J_{il}^{cs}$

and the total charge in the right hand side (see [EEM without shells](EEM-without-shells.ipynb)) is replaced by 

$q_{tot}-\sum\limits_{l=1}^N q_i^s$

Note that once the core charges are determined, the shell positions need to be minimized again. Both processes of determing core charges and optimization of shell positions need to be done in a self consistent fashion, as shown in the Python code below.

In [1]:
#!/usr/bin/env python3

import sys
import numpy as np
import math
from charge_utils import *
from test_systems import *

debug        = False

def solveEEM(names, coords, qtotal, qshell, model, verbose=False, shells=False):
    # Compute Coulomb
    J = calcJEEM(names, coords, model)
    N = coords.shape[0]
    # Tolerance for convergence
    toler     = 0.000001
    converged = False
    shellmin  = ShellMinimize(names, coords, qshell, model)
    q0 = np.zeros(N)
    for i in range(N):
        q0[i] = qtotal/N-qshell[i]
    chi = get_chi(model)
    eta = get_eta(model)
    chi2old = shellmin.minimize(q0)
    iter = 0
    while not converged and iter < 50:
        # First determine charges
        qstot = qtotal
        rhs   = np.zeros(N+1,dtype=float)
        for i in range(N):
            if names[i] in chi:
                rhs[i] = -chi[names[i]]
                if shells:
                    Jcs     = 0.5*calcJcs(i, names, coords, shellmin.shellX, qshell, model)
                    rhs[i] -= (eta[names[i]]*qshell[i] + Jcs)
                    qstot  -= qshell[i]
                    if debug:
                        print("i: %d %2s chi: %g 1/2 Jcs: %g eta*q: %g rhs: %g" % (i, names[i], chi[names[i]], Jcs, eta[names[i]]*qshell[i], rhs[i]))
            else:
                print("No chi for %s" % names[i])
                exit(1)
        rhs[N] = qstot
        q = np.linalg.solve(J, rhs)
        if debug:
            rhs2 = np.dot(J, q)
            print("rhs2 {}".format(rhs2))
        if shells:
            # Then minimize shells
            chi2 = shellmin.minimize(q)
            rmsd = shellmin.rmsd
            print("rmsd %10g energy %10g" % (rmsd, chi2))
            if abs(chi2-chi2old) < toler:
                converged = True
            else:
                chi2old = chi2
        else:
            converged = True
        if verbose or debug:
            print("J = \n{}".format(J))
            print("rhs = {}".format(rhs))
            print("q = {}".format(q))
            y = np.dot(J,q)
            print("y = {}".format(y))
        iter += 1
    return q, np.reshape(np.copy(shellmin.shellX), (-1,3))

def run_compound(molname, verbose, shells, model):
    mol = get_system(molname)
    if not mol:
        print("No test system %s defined" % molname)
    else:
        print("\n%s" % molname)
        q, shellX  = solveEEM(mol["names"], mol["coords"],
                                  mol["qtotal"], mol["qshell"], model, verbose, shells)
        for i in range(mol["coords"].shape[0]):
            print("q[%s] = %8g (including qshell %8g)" % (mol["names"][i], q[i], q[i]+mol["qshell"][i]))
        printDipole(q, mol["coords"], mol["atomnr"], shells, mol["qshell"], shellX)

verbose = False
shells = True

for compound in [ "methanol", "water", "acetate", "carbon-monoxide" ]:
    run_compound(compound, verbose, shells, "ACM-pg")



methanol
rmsd  0.0130687 energy   -204.065
rmsd 0.000675707 energy   -204.141
rmsd 4.59154e-05 energy   -204.145
rmsd 3.31214e-06 energy   -204.145
rmsd 2.09655e-07 energy   -204.145
rmsd          0 energy   -204.145
rmsd          0 energy   -204.145
q[oh] =  1.73678 (including qshell -0.263219)
q[hp] =  1.13181 (including qshell 0.131809)
q[c3] =   1.8484 (including qshell -0.151598)
q[h1] =  1.09276 (including qshell 0.0927589)
q[h1] =  1.09277 (including qshell 0.0927656)
q[h1] =  1.09748 (including qshell 0.0974831)
Dipole = 0.92174 Debye

water
rmsd  0.0108719 energy   -51.8545
rmsd 0.00017587 energy   -51.8918
rmsd 2.85822e-06 energy   -51.8924
rmsd          0 energy   -51.8924
rmsd          0 energy   -51.8924
q[hw] =   1.1323 (including qshell 0.132295)
q[ow] =  1.73541 (including qshell -0.264591)
q[hw] =   1.1323 (including qshell 0.132295)
Dipole = 0.538851 Debye

acetate
rmsd  0.0223487 energy   -283.726
rmsd 0.00239263 energy   -284.017
rmsd 0.000257987 energy   -284.049
